# Retrieval Augmented Generation (RAG) with LangChain

In [2]:
from dotenv import load_dotenv

load_dotenv(dotenv_path="environment")

import os
import tqdm as notebook_tqdm
import langchain
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.callbacks import StdOutCallbackHandler

print(f"LangChain version: {langchain.__version__}")



LangChain version: 0.3.18


In [3]:
text = os.path.join("data", "Alice_in_Wonderland.txt")

In [4]:
loader = TextLoader(file_path=text)

In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
data = loader.load_and_split(text_splitter=text_splitter)
data[2:5]

[Document(metadata={'source': 'data/Alice_in_Wonderland.txt'}, page_content="Title: Alice's Adventures in Wonderland\n\n\nAuthor: Lewis Carroll\n\nRelease date: June 27, 2008 [eBook #11]\n                Most recently updated: February 4, 2024\n\nLanguage: English\n\nCredits: Arthur DiBianca and David Widger\n\n\n*** START OF THE PROJECT GUTENBERG EBOOK ALICE'S ADVENTURES IN WONDERLAND ***\n[Illustration]\n\n\n\n\nAlice’s Adventures in Wonderland\n\nby Lewis Carroll\n\nTHE MILLENNIUM FULCRUM EDITION 3.0\n\nContents"),
 Document(metadata={'source': 'data/Alice_in_Wonderland.txt'}, page_content='CHAPTER I.     Down the Rabbit-Hole\n CHAPTER II.    The Pool of Tears\n CHAPTER III.   A Caucus-Race and a Long Tale\n CHAPTER IV.    The Rabbit Sends in a Little Bill\n CHAPTER V.     Advice from a Caterpillar\n CHAPTER VI.    Pig and Pepper\n CHAPTER VII.   A Mad Tea-Party\n CHAPTER VIII.  The Queen’s Croquet-Ground\n CHAPTER IX.    The Mock Turtle’s Story\n CHAPTER X.     The Lobster Quadrill

In [6]:
embeddings = OpenAIEmbeddings(api_key=os.getenv("OPENAI_API_KEY"), show_progress_bar=True)

/tmp/ipykernel_16968/3998137413.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(api_key=os.getenv("OPENAI_API_KEY"), show_progress_bar=True)


In [7]:
index = FAISS.from_documents(data, embeddings)

  0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
query = "Should people go back to yesterday?"
index.similarity_search_with_relevance_scores(query)

  0%|          | 0/1 [00:00<?, ?it/s]

[(Document(id='236f6557-c627-41a3-9709-00b3f2f96afb', metadata={'source': 'data/Alice_in_Wonderland.txt'}, page_content='“I could tell you my adventures—beginning from this morning,” said\nAlice a little timidly: “but it’s no use going back to yesterday,\nbecause I was a different person then.”\n\n“Explain all that,” said the Mock Turtle.\n\n“No, no! The adventures first,” said the Gryphon in an impatient tone:\n“explanations take such a dreadful time.”'),
  0.7084195461913874),
 (Document(id='3ea1bd89-6a4a-4c7a-a59b-627135ddc102', metadata={'source': 'data/Alice_in_Wonderland.txt'}, page_content='coming back to finish his story.'),
  0.6692098061860963),
 (Document(id='487973d3-6955-4384-b601-dab52d1bb397', metadata={'source': 'data/Alice_in_Wonderland.txt'}, page_content='“You ought to be ashamed of yourself for asking such a simple\nquestion,” added the Gryphon; and then they both sat silent and looked\nat poor Alice, who felt ready to sink into the earth. At last the\nGryphon said 

In [9]:
retriever = index.as_retriever()
retriever.search_kwargs['fetch_k'] = 20
retriever.search_kwargs['maximal_marginal_relevance'] = True
retriever.search_kwargs['k'] = 10

In [10]:
llm = ChatOpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [11]:
chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    verbose=True
)

handler = StdOutCallbackHandler()
response = chain.invoke({"query": query}, config={"callbacks": [handler]})
print(response["result"])



> Entering new RetrievalQA chain...


  0%|          | 0/1 [00:00<?, ?it/s]



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
“I could tell you my adventures—beginning from this morning,” said
Alice a little timidly: “but it’s no use going back to yesterday,
because I was a different person then.”

“Explain all that,” said the Mock Turtle.

“No, no! The adventures first,” said the Gryphon in an impatient tone:
“explanations take such a dreadful time.”

coming back to finish his story.

“You ought to be ashamed of yourself for asking such a simple
question,” added the Gryphon; and then they both sat silent and looked
at poor Alice, who felt ready to sink into the earth. At last the
Gryphon said to the Mock Turtle, “Drive on, old fellow! Don’t be all
day about it!” and he went on in these words:

“Yes, we went to school in th

In [12]:
query = "What path should you take if you don't know where you are going?"
response = chain.invoke({"query": query}, config={"callbacks": [handler]})
print(response["result"])



> Entering new RetrievalQA chain...


  0%|          | 0/1 [00:00<?, ?it/s]



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
“That depends a good deal on where you want to get to,” said the Cat.

“I don’t much care where—” said Alice.

“Then it doesn’t matter which way you go,” said the Cat.

“—so long as I get _somewhere_,” Alice added as an explanation.

“Oh, you’re sure to do that,” said the Cat, “if you only walk long
enough.”

Alice felt that this could not be denied, so she tried another
question. “What sort of people live about here?”

“The first thing I’ve got to do,” said Alice to herself, as she
wandered about in the wood, “is to grow to my right size again; and the
second thing is to find my way into that lovely garden. I think that
will be the best plan.”

There were doors all round the hall, but they were all 